In [1]:
from __future__ import print_function
from collections import deque
from tensorflow.python.framework import ops
from rl_reinforce import REINFORCEothello
from othello_net import *
from othello_rules import *
from feature_extractor import *
import os, random, sys
import tensorflow as tf
import time
from datetime import datetime
from example_states import *
from training_utils import *
from reinforcement_utils import *
import random
import numpy as np

In [2]:
test = policy_gradient_example()
test = np.transpose(test)
board = test
player = -1
test = board_to_input_c(test, -1, ['45', '67', '12', '41'])
tf.logging.set_verbosity(tf.logging.FATAL)
np.set_printoptions(precision=2)
tf.reset_default_graph()
tf_graph_1 = tf.Graph()
with tf_graph_1.as_default():
    img_1, train_step_1, adam_1, ground_truth_1, loss_1, pred_1, learn_rate_1, score_1 = create_no_graph()
sess_1 = tf.Session(graph=tf_graph_1)
features = [test]
with sess_1.as_default():
    with tf_graph_1.as_default():
        rl_reinforce_1 = REINFORCEothello(sess_1, adam_1, learn_rate_1, loss_1, score_1, ground_truth_1, img_1)
        tf.global_variables_initializer().run()
        saver_1 = tf.train.Saver(tf.global_variables(), max_to_keep=9999)
        model_1 = "../reinforcement/drill_1363.ckpt"
        #model_1 = "../supervised/models2/layers8filters64.ckpt"
        if os.path.isfile(model_1 + ".meta"):
            saver_1.restore(sess_1, model_1)
            print("loaded graph 1 as " + model_1)
prediction = sess_1.run(pred_1, feed_dict={img_1:features})
for sample in range(0, 10):
    sampled_move = sample_action(board, player, prediction)
    print(sampled_move)
prediction = np.squeeze(prediction)
print(prediction.shape)
prediction = np.transpose(prediction)
print(np.transpose(prediction[0]))
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(np.transpose(prediction[1]))
board = test
player = -1
label = move_to_label('15')
action = label
state = features
reward = 1
action = [action]
action = np.transpose(action)

loaded graph 1 as ../reinforcement/drill_1363.ckpt
14
14
14
14
15
14
14
14
14
14
(8, 8, 2)
[[ 1.    1.    1.    0.08  0.9   1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.  ]]
[[0.00 0.00 0.00 0.92 0.10 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]]


In [3]:
def print_last_ntest_fancy_board(logfile):
    lines = []
    cnt = 0
    for line in reversed(open(logfile).readlines()):
        current_line = line.rstrip()
        if current_line == '   A B C D E F G H':
            cnt = cnt+1
        #print(line.rstrip())
        lines.append(current_line)
        if cnt > 1:
            break
    board_height = len(lines)
    cleaned_lines = []
    numpy_board = np.zeros((8,8))
    for line in reversed(range(4, board_height-1)):
        current_line = lines[line]
        #print(current_line)
        current_line = str(current_line[3:-2])
        current_line = current_line.replace('O', '1')
        current_line = current_line.replace('-', '0')
        current_line = current_line.replace('*', '-1')
        cleaned_lines.append(current_line.split(" "))
    #print("cleaned lines length")
    #print(len(cleaned_lines))
    for i in range(8):
        for j in range(8):
            #print("numpy")
            numpy_board[i, j] = float(cleaned_lines[i][j])
    #print(numpy_board)
    return(numpy_board)
def convothello_move_to_ntest_move(convothello_move):
    
    if convothello_move[1] == '1':
        return 'A' + str(convothello_move[0])
    if convothello_move[1] == '2':
        return 'B' + str(convothello_move[0])
    if convothello_move[1] == '3':
        return 'C' + str(convothello_move[0])
    if convothello_move[1] == '4':
        return 'D' + str(convothello_move[0])
    if convothello_move[1] == '5':
        return 'E' + str(convothello_move[0])
    if convothello_move[1] == '6':
        return 'F' + str(convothello_move[0])
    if convothello_move[1] == '7':
        return 'G' + str(convothello_move[0])
    if convothello_move[1] == '8':
        return 'H' + str(convothello_move[0])
    return 'X0'

def end_of_game_prints(board):
    print("Game over dawg!")
    print("Black pieces: " + str(get_points(board, -1)))
    print("White pieces: " + str(get_points(board, 1)))
    print("winner is: " + str(get_winner(board, -1, 1)))
    #print(board)
    print("\n")
    return 0

In [12]:
#MAIN TESTING CELL NOW
import os
os.chdir('/home/hlynurd/Notebook/Othello/analysis/')
from __future__ import print_function
from collections import deque
from tensorflow.python.framework import ops
from rl_reinforce import REINFORCEothello
from othello_net import *
from othello_rules import *
from feature_extractor import *
import random, sys
import tensorflow as tf
import time
from datetime import datetime
from example_states import *
from training_utils import *
from reinforcement_utils import *
import random
import numpy as np
import pexpect
np.set_printoptions(precision=2)

tf.reset_default_graph()
tf_graph_1 = tf.Graph()
with tf_graph_1.as_default():
    img_1, train_step_1, adam_1, ground_truth_1, loss_1, pred_1, learn_rate_1, score_1 = create_no_graph()
sess_1 = tf.Session(graph=tf_graph_1)
with sess_1.as_default():
    with tf_graph_1.as_default():
        rl_reinforce_1 = REINFORCEothello(sess_1, adam_1, learn_rate_1, loss_1, score_1, ground_truth_1, img_1)
        tf.global_variables_initializer().run()
        saver_1 = tf.train.Saver(tf.global_variables(), max_to_keep=9999)
        model_1 = "../reinforcement/drill_1363.ckpt"
        #model_1 = "../supervised/models2/layers8filters64.ckpt"
        if os.path.isfile(model_1 + ".meta"):
            saver_1.restore(sess_1, model_1)
            print("loaded graph 1 as " + model_1)




competition_rounds = 20
hang_count = 0
wins = 0
for comp in range(competition_rounds):
    print(comp)
    os.chdir('/home/hlynurd/Documents/ntest/ntest/build/')
    child2 = pexpect.spawn('./ntest gh1 a0.0001 m0.0001 ') 
    #child2 = pexpect.spawn('./ntest gh1 a1 ') 
    child2.timeout = 25
    child2.logfile = open("/tmp/mylog2", "w+")
    child2.expect ('Please enter next move:')
    print_last_ntest_fancy_board('/tmp/mylog2')
    sampled_move = 56
    previous_moves = [11, 22, 33, 44]
    total_start = start = datetime.now()
    my_thinking_times = datetime.now() - datetime.now()
    turns = 0
    timedout = 0
    hang_count = 0
    while True:
        #print(turns)
        turns = turns + 1
        if turns > 15:
            child2.sendline('ap')
            board = print_last_ntest_fancy_board('/tmp/mylog2')
            winner = get_winner(board, -1, 1)
            if winner == -1:
                wins = wins + 1
            break
        player = -1
        start = datetime.now()
        board = print_last_ntest_fancy_board('/tmp/mylog2')
        start2 = datetime.now()
        features = board_to_input_c(board, player, previous_moves)
        end2 = datetime.now()
        #print("time to calculate features: " + str(end2-start2))
        legal_moves = find_legal_moves_c(board, player)
        if len(legal_moves) > 1:
            prediction = sess_1.run(pred_1, feed_dict={img_1:[features]})
            sampled_move = sample_action(board, player, prediction)
        elif len(legal_moves) == 1:
            sampled_move = legal_moves[0]        
        else: 
            sampled_move = 'Pass'
        if sampled_move == previous_moves[1]:
            sampled_move = legal_moves[0]
        my_legal_moves = find_legal_moves_c(board, -1)
        
        #print("My prev move was: " + str(sampled_move))
        if sampled_move == 'Pass':
            next_move = sampled_move
            hang_count = hang_count + 1
            legal_moves_white = find_legal_moves_c(board, 1)
            if len(legal_moves_white) <= 1 and hang_count > 0:
                while len(legal_moves_white) >= 1:
                    board = make_move_c(board, legal_moves_white[0], 1)
                    legal_moves_white = find_legal_moves_c(board, 1)
                end_of_game_prints(board)
                break
            turns = turns - 1
        else:
            next_move = convothello_move_to_ntest_move(str(sampled_move))
            prev_move = sampled_move
            previous_moves[3] = previous_moves[1]
            previous_moves[1] = sampled_move
        end = datetime.now()
        #print("total own time: " + str(end - start))
        my_thinking_times = my_thinking_times + end - start
        child2.sendline(next_move)
        try:
            child2.expect ('Please enter next move:')
        except:
            legal_moves_white = (find_legal_moves_c(board, 1))
            hang_count = hang_count + 1
            if hang_count > 0:
                while len(legal_moves_white) >= 1:
                    board = make_move_c(board, legal_moves_white[0], 1)
                    legal_moves_white = find_legal_moves_c(board, 1)
                end_of_game_prints(board)
                break
    #print("my total thinking time: " + str(my_thinking_times))
    #print("current game time: " + str(end - total_start))
print(str(float(wins) / competition_rounds))
                  

loaded graph 1 as ../reinforcement/drill_1363.ckpt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.6
